# 一、载入作业所需要使用的库

In [1]:
import sys
import pandas as pd
import csv
import numpy as np
import collections
import os
import re
import jieba
import string
import operator
from collections import Counter,defaultdict
from gensim import corpora
import tempfile
from gensim.models import ldaseqmodel
from gensim import corpora, models,similarities
import logging
from gensim.models.wrappers.dtmmodel import DtmModel
from gensim.corpora import Dictionary, bleicorpus
import pyLDAvis
import gensim  
from gensim import models  
import pyLDAvis.gensim 
from gensim.models import LdaModel

C:\Users\Bokkin Wang\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# 二、设置工作路径读取数据

In [2]:
os.chdir('D:/bigdatahw/应用统计案例/case2018Lab9')  #设置工作路径

- 导进去的文档是按照时间顺序从早到晚排列，也就是说，df中最开始的是4月17日的文档，最末尾的是5月19日的文档。下面的clean_text也是按顺序的，这一点对之后建立动态主题模型很重要。

In [3]:
df = pd.read_csv("alldata.csv",encoding = "gb18030")
df.head(3)

,title,date,time,source,abstract,detail,url,origin_url,period
0,"通源石油:中美贸易战对公司利大于弊,油价复苏带动业绩大增",2018/3/28,15:34:07,中证网,"事件:公司3月26日发布关于美贸易政策影响的说明公告,认为中美贸易战对公司的营业收入和日常经...","事件:公司3月26日发布关于美贸易政策影响的说明公告,认为中美贸易战对公司的营业收入和日常经...",http://finance.sina.com.cn/roll/2018-03-28/doc...,http://www.cs.com.cn/gppd/tzpj/201803/t2018032...,1
1,人民日报海外版:美国一意孤行 世界一声叹息,2018/3/29,5:22:07,人民日报海外版-海外网,欧洲股市、亚太股市同样无法幸免。“中美贸易战还未真正开打，全球股市已经出现大幅下挫，这就足以...,原标题：美国一意孤行 世界一声叹息（环球热点）美国特朗普政府掀起的贸易保护阴云弥漫不散。3...,http://news.sina.com.cn/c/nd/2018-03-29/doc-if...,http://news.haiwainet.cn/n/2018/0329/c3541083-...,1
2,分析：中美贸易关系将如何影响国内外豆粕行情？,2018/3/29,8:13:42,农产品期货网,00:40分左右，美国总统特朗普正式签署了制裁中国的总统备忘录，美国政府计划对中国的至少60...,?? 3月初以来芝商所旗下CBOT期价（代码：ZMK8）一度陷入震荡回落囧境，最低曾下探至...,http://finance.sina.com.cn/money/future/agri/2...,http://www.ncpqh.com/news/getDetail?newsclass=...,1


- 统计各个时期的新闻数量

In [4]:
df["date"]= pd.to_datetime(df["date"],format="%Y-%m-%d")
df["date"].value_counts(sort = True)

2018-04-09    90
2018-04-10    60
2018-04-06    44
2018-04-11    39
2018-04-05    37
2018-04-08    35
2018-04-18    29
2018-04-07    28
2018-04-17    27
2018-04-13    22
2018-05-08    14
2018-05-15    13
2018-04-02    13
2018-04-27    12
2018-04-19    12
2018-04-04    11
2018-04-24    11
2018-04-23    10
2018-04-12     9
2018-05-02     9
2018-04-03     7
2018-05-07     6
2018-05-14     5
2018-05-03     5
2018-04-22     5
2018-05-20     5
2018-05-04     5
2018-04-25     4
2018-03-31     4
2018-03-30     4
2018-05-10     4
2018-04-21     3
2018-05-16     3
2018-05-18     3
2018-05-01     3
2018-04-15     2
2018-05-11     2
2018-04-28     2
2018-04-16     2
2018-03-29     2
2018-05-05     2
2018-03-28     1
2018-04-01     1
2018-05-09     1
2018-05-17     1
2018-04-14     1
2018-04-26     1
2018-05-19     1
Name: date, dtype: int64

# 三、新闻文本预处理

- 文本处理函数

In [5]:
def get_text(data):
    text=data["detail"]
    text = text.dropna() 
    len(text)
    text=[t.encode('utf-8').decode("utf-8") for t in text] 
    return text

def get_stop_words(file='stopWord.txt'):
    file = open(file, 'rb').read().decode('utf8').split(',')
    file = [line.strip() for line in file]
    return set(file)                                         #查分停用词函数

def rm_tokens(words):                                        # 去掉一些停用词和完全包含数字的字符串
    words_list = list(words)
    stop_words = get_stop_words()
    for i in range(words_list.__len__())[::-1]:
        if words_list[i] in stop_words:                      # 去除停用词
            words_list.pop(i)
        elif words_list[i].isdigit():
            words_list.pop(i)
    return words_list

def rm_char(text):

    text = re.sub('\x01', '', text)                        #全角的空白符
    text = re.sub('\u3000', '', text) 
    text = re.sub(r"[\)(↓%·▲ \s+】【]","", text.encode("utf-8").decode("utf-8")) 
    text = re.sub(r"[\n\da-z+（）《》↗><‘’“”.-]","",text,flags=re.I)
    text = re.sub('\n+', " ", text)
    text = re.sub('[，、：。！？?；——]', " ", text)
#    text = re.sub(' +', "E S", text)
    return text

def convert_doc_to_wordlist(str_doc, cut_all=False):
    sent_list = str_doc.split('\n')
    sent_list = map(rm_char, sent_list)                       # 去掉一些字符，例如\u3000
    word_2dlist = [rm_tokens(jieba.cut(part, cut_all=cut_all))
                   for part in sent_list]                     # 分词
    word_list = sum(word_2dlist, [])
    rm_space = [w for w in word_list if w not in [""," "]]
    return rm_space

def rm_1ow_freq_word(texts,low_freq=1):
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    
    texts = [[token for token in text if frequency[token] > low_freq] for text in texts]
    return texts

- 增加词库分词

In [6]:
jieba.add_word("中美")
jieba.add_word("中美贸易战")
jieba.add_word("习主席")
jieba.add_word("中国商务部")
jieba.add_word("拒绝令")
jieba.add_word("不公平")
jieba.add_word("推文")
jieba.add_word("推特")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BOKKIN~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.687 seconds.
Prefix dict has been built succesfully.


In [7]:
clean_text=[convert_doc_to_wordlist(line) for line in get_text(df)] #去除停用词、标点、数字、特殊字符、分词
clean_text2 = rm_1ow_freq_word(clean_text)                          #去除只出现过一次的词

In [8]:
"/".join(clean_text2[302])

'上/交易日/市况/回顾/周一/美元/竞对/货币/小幅/走低/此前/美元/连续/两周/上涨/投资者/解除/一些/美元/空头/押注/贸易战/忧虑/依然/存在/这令/美元/整体/承压/欧元/欧洲央行/行长/德拉/基/讲话/后/上涨/市场/认为/德拉/基的/讲话/欧元/起到/支撑/作用/大宗/商品/股市/方面/贵金属/小幅/收涨/国际/油价/收涨/逾/欧美/股市/涨跌/不/交易日/市场/重点/关注/中国/国家/主席/习近平/博鳌/亚洲/论坛/主旨/演讲/美国/数据/外汇/欧元/美元/收报/英镑/美元/收报/澳元/美元/收报/美元/日元/收报/美元/加元/收报/美元/瑞郎/收报/大宗/商品/现货/金/收报/美元/盎司/期金/收报/美元/盎司/现货/银/收报/美元/盎司/期银/收报/美元/盎司/布伦特/收报/美元/桶/原油/收报/美元/桶/欧美/股市/周一/美市/盘中/最后/一个/小时/亚马逊/波音/翻盘/美股/急转直下/道/指点/涨势/几乎/回吐/殆尽/股市/收盘/远/低于/盘中/高点/交易员/表示/一份/官员/突击/搜查/美国/总统/特朗普/私人/律师/办公室/报告/加剧/尾盘/跌势/道琼斯/工业/平均/指数/上涨/点/,/点/默克/英特尔/指数/中/表现/最好/股票/标准/普尔/指数/上涨/收于/,/点/科技/板块/大涨/纳斯达克/综合/指数/上涨/至点/欧洲/股市/周一/小幅/收高/因在/周二/关键/讲话/前/市场/中美贸易战/担忧/略有/减弱/泛欧/斯托克/指数/徘徊/平盘/附近/收盘/仅/上涨/板块/不同/方向/移动/当天/早些时候/指数/一直/上涨/汇市/上/交易日/纽约/时段/欧元/美元/开于/时段/内/窄幅/震荡/尾盘/交投/附近/技术/面看/绿色/动能/柱/稳步/收缩/指标/指向/中线/上方/预示/汇价/下/行动/正在/消退/汇价/上行/初步/阻力/进一步/阻力/位于/更/关键/阻力/位于/汇价/下行/初步/支撑/位于/进一步/支撑/位于/更/关键/支撑/位于/英镑/美元/时段/开于/时段/内/小幅/下挫/尾盘/交投/附近/技术/面看/红色/动能/柱/显现/指标/低位/向上/进入/超买/区域/显示/汇价/上行/势头/依然/汇价/上行/初步/阻力/位于/进一步/阻力/位于/更/关键/阻力/位于/汇价/下行/初步/支撑/位于/进一步/支撑/位于/更/关键/支撑

# 四、动态主题模型
- gensim：一个python的专门做主题模型的库，它的官方介绍是：Topic modelling for humans

### 4.1 利用分好词的文本建立gensim格式的字典
- 字典格式：**{词：索引号}** 即一个词有对应一个索引
- 可以看到字典中一共有14766 个词

In [9]:
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))
dictionary = corpora.Dictionary(clean_text2)
dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))  # store the dictionary, for future reference
print(dictionary)
print(dictionary.token2id)

Folder "C:\Users\BOKKIN~1\AppData\Local\Temp" will be used to save temporary dictionary and corpus.
Dictionary(14766 unique tokens: [',', ':', '一体化', '一倍', '上']...)
{',': 0, ':': 1, '一体化': 2, '一倍': 3, '上': 4, '上半年': 5, '下属': 6, '不': 7, '不会': 8, '不到': 9, '业务': 10, '业绩': 11, '中': 12, '中枢': 13, '中美': 14, '中美贸易战': 15, '中长期': 16, '为元': 17, '买入': 18, '事件': 19, '产量': 20, '人': 21, '仍': 22, '仍存': 23, '以来': 24, '低于': 25, '低谷': 26, '供应商': 27, '保护': 28, '倍': 29, '元': 30, '全部': 31, '公司': 32, '公司业绩': 33, '公告': 34, '具备': 35, '再次': 36, '减少': 37, '分别': 38, '北美': 39, '原油': 40, '反弹': 41, '发展': 42, '发布': 43, '受': 44, '受益': 45, '员工': 46, '国内': 47, '国际': 48, '均': 49, '均价': 50, '基础': 51, '增长': 52, '复苏': 53, '多名': 54, '大增': 55, '大幅': 56, '子公司': 57, '已': 58, '带动': 59, '弹性': 60, '强力': 61, '影响': 62, '快速': 63, '成立': 64, '成长': 65, '扩大': 66, '投资': 67, '持续': 68, '控股': 69, '提示': 70, '收入': 71, '改善': 72, '政策': 73, '新高': 74, '日常': 75, '时': 76, '显著': 77, '景气': 78, '最新': 79, '未来': 80, '本土': 81, '本地': 82, '来看': 83, '来自': 8

### 4.2 利用分好词的文本建立gensim格式的语料库
- 可以看到一共有610条文本
- 语料库格式：**(词索引，词出现的次数)**

In [10]:
corpus = [dictionary.doc2bow(text) for text in clean_text2]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus)  # store to disk, for later use
len(corpus)

610

In [16]:
corpus[0]  #展示第1条新闻文本LDA处理结果

[(0, 26),
 (1, 3),
 (2, 1),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 3),
 (11, 3),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 3),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 3),
 (30, 2),
 (31, 1),
 (32, 17),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 2),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 3),
 (45, 3),
 (46, 1),
 (47, 1),
 (48, 2),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 5),
 (53, 3),
 (54, 1),
 (55, 2),
 (56, 4),
 (57, 2),
 (58, 1),
 (59, 2),
 (60, 1),
 (61, 1),
 (62, 4),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 2),
 (69, 1),
 (70, 1),
 (71, 3),
 (72, 1),
 (73, 3),
 (74, 1),
 (75, 1),
 (76, 1),
 (77, 2),
 (78, 1),
 (79, 1),
 (80, 3),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 2),
 (86, 4),
 (87, 2),
 (88, 3),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 2),
 (96, 1),
 (97, 1),
 (98, 1),
 (99, 3),
 (100, 1

### 4.3 建立动态主题模型
- time_slice是一个列表，指定每个时间段的文本数。如下所示，第一个时间段有32篇新闻，第二个时间段有378篇新闻......第五个时间段有31篇新闻。因此语料库按照时间顺序排好很重要

In [11]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
time_slice = list(df["period"].value_counts(sort=False))
time_slice

[32, 378, 135, 34, 31]

- ldaseqmodel.LdaSeqModel，建立动态主题模型，即加入时间因素，可使不同主题随着时间变动
- 因为主题为中美贸易战事件，只有一个，设置num_topics=1
- 考虑到运行时间，设置lda迭代次数为15，EM算法迭代次数为10
- 为使不同时间段的主题差异度大一些，设置chain_variance=50

In [ ]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, 
                                 time_slice=time_slice, num_topics=3,
                                 lda_inference_max_iter=15,
                                 em_max_iter=1,
                                 chain_variance=50)

2018-06-04 00:37:32,387 : INFO : using symmetric eta at 0.3333333333333333
2018-06-04 00:37:32,390 : INFO : using serial LDA version on this node
2018-06-04 00:37:32,394 : INFO : running online (multi-pass) LDA training, 3 topics, 10 passes over the supplied corpus of 610 documents, updating model once every 610 documents, evaluating perplexity every 610 documents, iterating 50x with a convergence threshold of 0.001000
2018-06-04 00:37:33,670 : INFO : -10.093 per-word bound, 1092.5 perplexity estimate based on a held-out corpus of 610 documents with 345053 words
2018-06-04 00:37:33,672 : INFO : PROGRESS: pass 0, at document #610/610
2018-06-04 00:37:34,254 : INFO : topic #0 (0.010): 0.020*"美国" + 0.011*"中国" + 0.007*"市场" + 0.005*"大豆" + 0.005*"贸易" + 0.005*"影响" + 0.004*"美元" + 0.004*"关税" + 0.004*"经济" + 0.004*"表示"
2018-06-04 00:37:34,255 : INFO : topic #1 (0.010): 0.014*"中国" + 0.012*"美国" + 0.010*"市场" + 0.005*"经济" + 0.005*"上" + 0.005*"贸易" + 0.004*"影响" + 0.004*"可能" + 0.004*"中" + 0.004*"中美"
201

2018-06-04 00:37:48,086 : INFO : topic #2 (0.010): 0.029*"中国" + 0.026*"美国" + 0.009*"贸易" + 0.008*"特朗普" + 0.006*"贸易战" + 0.006*"关税" + 0.005*"中美" + 0.005*"中" + 0.005*"经济" + 0.005*"出口"
2018-06-04 00:37:48,087 : INFO : topic diff=0.051213, rho=0.301511
2018-06-04 00:37:50,623 : INFO :  EM iter 0
2018-06-04 00:37:50,624 : INFO : E Step
2018-06-04 00:37:50,626 : INFO : using symmetric eta at 0.3333333333333333
2018-06-04 00:37:50,629 : INFO : using serial LDA version on this node
2018-06-04 00:38:44,825 : INFO : M Step
2018-06-04 00:38:44,826 : INFO : Fitting topic number 0
2018-06-04 00:38:45,618 : INFO : Computing bound, all times
2018-06-04 00:38:46,147 : INFO : initial sslm bound is 17923028.125581
2018-06-04 00:39:10,920 : INFO : Computing bound, all times
2018-06-04 00:39:11,431 : INFO : iteration 1 iteration lda seq bound is 17955224.735618 convergence is 0.001796
2018-06-04 00:39:31,460 : INFO : Computing bound, all times
2018-06-04 00:39:31,985 : INFO : iteration 2 iteration lda seq b

KeyboardInterrupt: 

2018-06-04 00:44:56,565 : ERROR : Uncaught exception, closing connection.
Traceback (most recent call last):
  File "C:\Users\Bokkin Wang\Anaconda3\lib\site-packages\zmq\eventloop\zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\Bokkin Wang\Anaconda3\lib\site-packages\tornado\stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\Bokkin Wang\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "C:\Users\Bokkin Wang\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "C:\Users\Bokkin Wang\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "C:\Users\Bokkin Wang\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 637, in _abort_queues
    self._abort_queue(stream)
  File "C:\Users\Bokkin Wang\Anacon

- #最近一段时间的主题词

In [18]:
ldaseq.print_topics(time=4)

NameError: name 'ldaseq' is not defined

- 输出同一个主题在所有时间段的主题词

In [38]:
ldaseq.print_topic_times(topic=0, top_terms=15)

[[('公司', 0.012975673104562232),
  (',', 0.008019866946886613),
  ('业务', 0.0078558536234208318),
  ('亿元', 0.0073081180760922859),
  ('中信', 0.0072157609530275847),
  ('美元', 0.0068300437084578254),
  ('美国', 0.0063476107448551733),
  ('建投', 0.0056467362400548842),
  ('黄金', 0.0056216228240632818),
  ('市场', 0.0055811968127323216),
  ('证券', 0.0055390613674058181),
  ('油价', 0.0052834667143776323),
  ('券商', 0.0050645464366605381),
  ('中', 0.0050371846983716549),
  ('可能', 0.004339157672191494)],
 [('美元', 0.013086926397623561),
  ('美国', 0.010737953835322184),
  ('市场', 0.010520377177449554),
  ('黄金', 0.010000440237436573),
  ('数据', 0.0060744062601368063),
  ('上', 0.005765604428095439),
  ('上涨', 0.0051858580961365057),
  ('非农', 0.0048911517441525771),
  ('仍', 0.0047466740557897047),
  ('预期', 0.004740099745526913),
  ('可能', 0.0046500245418295864),
  ('支撑', 0.0045209871201241777),
  ('经济', 0.0044923608886999989),
  ('继续', 0.0043043081967587609),
  ('贸易', 0.0042095992642365498)],
 [('市场', 0.0125032101

### 4.4 找出在不同时间段波动较大的词
- 这些词最能体现不同时间段的特点

In [39]:
#保存所有时间段的前2000个主题词
topwords = ldaseq.print_topic_times(topic=0, top_terms=2000) 

#找5个时期，概率波动最大的词
word_pro = {}#保存词各个时间段的概率的字典
for tw in topwords:
    for w,p in tw:
#        print(w,p)
        if w not in word_pro:
            word_pro[w] = [p]
        else:
            word_pro[w].append(p)


word_var = {}#计算词的概率的标准差的字典
for w,p_list in word_pro.items():
    if len(p_list)==5:
        word_var[w]=np.std(p_list)
        
sorted_word_var = sorted(word_var.items(), key = operator.itemgetter(1), reverse=True) #=True 降序排列
sorted_word_var[:50]

[('美元', 0.0057283415879593241),
 ('公司', 0.0049048776518695802),
 ('美国', 0.0032567854358535636),
 ('支撑', 0.0031614500795238429),
 (',', 0.0030327211276269324),
 ('美股', 0.0028841759684766492),
 ('附近', 0.0028777255293910555),
 ('美联储', 0.0027894018141201571),
 ('亿元', 0.0026779486738091733),
 ('指数', 0.0026210352094999502),
 ('市场', 0.0025570193506170877),
 ('汇价', 0.0023159837754091772),
 ('黄金', 0.0022789999490939474),
 ('预期', 0.0020300407786548067),
 ('证券', 0.0020061485719475285),
 ('非农', 0.0019916816320887558),
 ('表示', 0.0019218192269641586),
 ('会议', 0.00187094368887709),
 ('加息', 0.0018477846694424031),
 ('资金', 0.0018202641059284943),
 ('油价', 0.0017787714696590412),
 ('维持', 0.0017423410270147733),
 ('此次', 0.001652432951691662),
 ('昨日', 0.001640795427373785),
 ('经济', 0.0015935535305094041),
 ('利率', 0.0015894182002552943),
 ('关注', 0.0015659450857442292),
 ('亿', 0.0015353002564788311),
 ('数据', 0.0015314858096434508),
 ('攀升', 0.0014932718781659905),
 ('央行', 0.0014579190179015305),
 ('中美', 0.001

In [22]:
#把波动比较大的前100个词在5个时间段的概率拎出来

var_word = [item[0] for item in sorted_word_var[:100] if len(item[0])>1]

word_pro_df = pd.DataFrame()
for w in var_word:
    onedf = pd.DataFrame({"word":[w],"p":[" ".join([str(p) for p in word_pro[w]])]})
    word_pro_df = word_pro_df.append(onedf)
word_pro_df
word_pro_df.to_csv("word_pro.csv",index=False)

### 4.5 LDA主题模型结果可视化


In [52]:
dtm_model = ldaseq #载入模型

In [53]:
doc_topic, topic_term, doc_lengths, term_frequency, vocab = dtm_model.dtm_vis(time=0, corpus=corpus)
vis_wrapper = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.display(vis_wrapper)
pyLDAvis.save_html(vis_wrapper, 'lda.html')  

C:\Users\Bokkin Wang\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


In [54]:
doc_topic, topic_term, doc_lengths, term_frequency, vocab = ldaseq.dtm_vis(time=0, corpus=corpus)
vis_dtm = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.display(vis_dtm)
pyLDAvis.save_html(vis_dtm, 'lda.html')

C:\Users\Bokkin Wang\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


In [12]:
lda = LdaModel(corpus=corpus,num_topics=3)
data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)

2018-06-04 01:04:22,251 : WARNING : no word id mapping provided; initializing from corpus, assuming identity
2018-06-04 01:04:22,270 : INFO : using symmetric alpha at 0.3333333333333333
2018-06-04 01:04:22,271 : INFO : using symmetric eta at 0.3333333333333333
2018-06-04 01:04:22,274 : INFO : using serial LDA version on this node
2018-06-04 01:04:22,285 : INFO : running online (single-pass) LDA training, 3 topics, 1 passes over the supplied corpus of 610 documents, updating model once every 610 documents, evaluating perplexity every 610 documents, iterating 50x with a convergence threshold of 0.001000
2018-06-04 01:04:22,286 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-06-04 01:04:23,608 : INFO : -10.087 per-word bound, 1087.7 perplexity estimate based on a held-out corpus of 610 documents with 345053 words
2018-06-04 01:04:23,609 : INFO : PROGRESS: pass 0, at document #610/610
2018-06-04 01:04

In [ ]:
pyLDAvis.show(data)
pyLDAvis.save_html(data, 'lda.html')  

In [ ]:
pyLDAvis.save_html(data, 'lda.html')  

# 五、lsi主题模型

- 将语料库的词频向量转换为tfidf向量

In [23]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

2018-06-03 21:58:38,577 : INFO : collecting document frequencies
2018-06-03 21:58:38,578 : INFO : PROGRESS: processing document #0
2018-06-03 21:58:38,620 : INFO : calculating IDF weights for 610 documents and 14765 features (186414 matrix non-zeros)


- 建立lsi模型

In [24]:
lsi_model = models.LsiModel(corpus=corpus_tfidf,id2word=dictionary,
            num_topics=5)

2018-06-03 21:58:40,725 : INFO : using serial LSI version on this node
2018-06-03 21:58:40,727 : INFO : updating model with new documents
2018-06-03 21:58:41,231 : INFO : preparing a new chunk of documents
2018-06-03 21:58:41,262 : INFO : using 100 extra samples and 2 power iterations
2018-06-03 21:58:41,262 : INFO : 1st phase: constructing (14766, 105) action matrix
2018-06-03 21:58:41,303 : INFO : orthonormalizing (14766, 105) action matrix
2018-06-03 21:58:41,564 : INFO : 2nd phase: running dense svd on (105, 610) matrix
2018-06-03 21:58:41,591 : INFO : computing the final decomposition
2018-06-03 21:58:41,593 : INFO : keeping 5 factors (discarding 76.089% of energy spectrum)
2018-06-03 21:58:41,598 : INFO : processed documents up to #610
2018-06-03 21:58:41,600 : INFO : topic #0(5.194): 0.199*"黄金" + 0.164*"大豆" + 0.146*"美元" + 0.099*"美国" + 0.092*"特朗普" + 0.091*"中国" + 0.091*"非农" + 0.089*"金价" + 0.085*"关税" + 0.078*"万吨"
2018-06-03 21:58:41,601 : INFO : topic #1(3.587): 0.362*"黄金" + -0.321

In [25]:
#这就是基于SVD建立的两个主题模型内容
lsiout=lsi_model.print_topics(5)
print (lsiout[0])
print (lsiout[1])
print (lsiout[2])
print (lsiout[3])
print (lsiout[4])

2018-06-03 21:58:44,280 : INFO : topic #0(5.194): 0.199*"黄金" + 0.164*"大豆" + 0.146*"美元" + 0.099*"美国" + 0.092*"特朗普" + 0.091*"中国" + 0.091*"非农" + 0.089*"金价" + 0.085*"关税" + 0.078*"万吨"
2018-06-03 21:58:44,281 : INFO : topic #1(3.587): 0.362*"黄金" + -0.321*"大豆" + 0.188*"美元" + 0.166*"金价" + 0.151*"非农" + -0.129*"万吨" + 0.113*"一线" + -0.106*"豆粕" + 0.105*"均线" + -0.095*"出口"
2018-06-03 21:58:44,282 : INFO : topic #2(3.171): 0.418*"大豆" + 0.205*"万吨" + 0.188*"豆粕" + 0.146*"吨" + 0.134*"美豆" + -0.133*"中兴" + 0.112*"豆油" + -0.108*"特朗普" + 0.102*"巴西" + 0.099*"价格"
2018-06-03 21:58:44,284 : INFO : topic #3(2.772): 0.231*"港股" + 0.210*"板块" + 0.206*"股" + -0.149*"黄金" + 0.131*"创业板" + -0.130*"特朗普" + 0.119*"报点" + 0.118*"逾" + -0.107*"台湾" + 0.105*"个股"
2018-06-03 21:58:44,285 : INFO : topic #4(2.473): 0.293*"盘面" + 0.211*"空单" + 0.194*"呈" + 0.178*"吨" + 0.175*"反弹" + -0.161*"大豆" + 0.156*"伦铜" + 0.156*"抢反弹" + 0.156*"尚存" + 0.152*"位置"


(0, '0.199*"黄金" + 0.164*"大豆" + 0.146*"美元" + 0.099*"美国" + 0.092*"特朗普" + 0.091*"中国" + 0.091*"非农" + 0.089*"金价" + 0.085*"关税" + 0.078*"万吨"')
(1, '0.362*"黄金" + -0.321*"大豆" + 0.188*"美元" + 0.166*"金价" + 0.151*"非农" + -0.129*"万吨" + 0.113*"一线" + -0.106*"豆粕" + 0.105*"均线" + -0.095*"出口"')
(2, '0.418*"大豆" + 0.205*"万吨" + 0.188*"豆粕" + 0.146*"吨" + 0.134*"美豆" + -0.133*"中兴" + 0.112*"豆油" + -0.108*"特朗普" + 0.102*"巴西" + 0.099*"价格"')
(3, '0.231*"港股" + 0.210*"板块" + 0.206*"股" + -0.149*"黄金" + 0.131*"创业板" + -0.130*"特朗普" + 0.119*"报点" + 0.118*"逾" + -0.107*"台湾" + 0.105*"个股"')
(4, '0.293*"盘面" + 0.211*"空单" + 0.194*"呈" + 0.178*"吨" + 0.175*"反弹" + -0.161*"大豆" + 0.156*"伦铜" + 0.156*"抢反弹" + 0.156*"尚存" + 0.152*"位置"')


- 将文章投影到主题空间中

In [26]:
corpus_lsi = lsi_model[corpus_tfidf]
for doc in corpus_lsi:
    print (doc)

[(0, 0.11982953307627903), (1, 0.00615671997800265), (2, 0.0045580508822347541), (3, 0.063311815061882853), (4, 0.10954246220653809)]
[(0, 0.23774595736407467), (1, -0.14280800278316474), (2, -0.187621533567191), (3, -0.14750870246895242), (4, 0.030055740986373706)]
[(0, 0.32783823121523414), (1, -0.27361591192099904), (2, 0.42929380151981189), (3, -0.0066231396115833518), (4, -0.05828603515596277)]
[(0, 0.10229788176861712), (1, -0.038804812419068982), (2, -0.041916529652513454), (3, 0.013557062468801003), (4, -0.0084653786578015415)]
[(0, 0.047117999397645653), (1, -0.030569037072002668), (2, -0.078136259020967908), (3, -0.089041096943237641), (4, 0.021555075883064847)]
[(0, 0.14768281625493451), (1, -0.10687825465180291), (2, -0.0086042938504498137), (3, -0.010738554653728895), (4, 0.0033435202979881489)]
[(0, 0.079056732788337455), (1, -0.049270673670841841), (2, -0.10133654473476668), (3, -0.097287094289690013), (4, 0.024970255424625776)]
[(0, 0.16500242531294301), (1, -0.11356835

[(0, 0.12621399124737209), (1, -0.009154583181309895), (2, -0.058911687170671488), (3, 0.19353869200095622), (4, -0.022828624834887942)]
[(0, 0.23307084228812697), (1, -0.060759608244545307), (2, -0.090275077853541677), (3, 0.19400323508657841), (4, 0.017428156398152744)]
[(0, 0.11412450292940253), (1, -0.022370225403586484), (2, -0.051762680179746605), (3, 0.097439496155668284), (4, -0.016639429654565781)]
[(0, 0.092301023027382342), (1, -0.053212116719934838), (2, -0.082487166007497559), (3, 0.046964517451198763), (4, -0.017160397605218324)]
[(0, 0.2021624551230988), (1, -0.12082540488262751), (2, -0.18799842981590825), (3, 0.1423974363371765), (4, -0.056990873410724542)]
[(0, 0.20434979615113377), (1, -0.012575642117103035), (2, 0.12843067460317342), (3, 0.087983558297281272), (4, 0.93637421532099396)]
[(0, 0.20264966741458731), (1, -0.0093983670946843102), (2, -0.12033080362898661), (3, 0.26802301546366714), (4, -0.032339834510255835)]
[(0, 0.13366027876125111), (1, -0.082937488837

- 输入一句话，查询属于LSI得到的哪个主题类型，先建立索引：

In [27]:
index = similarities.MatrixSimilarity(lsi_model[corpus])
query = "多家企业协会致信劝阻在特朗普签署总统备忘录的前一周"
query_bow = dictionary.doc2bow(list(jieba.cut(query)))
print (query_bow)
query_lsi = lsi_model[query_bow]
print (query_lsi)

2018-06-03 21:58:51,574 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-06-03 21:58:51,635 : INFO : creating matrix with 610 documents and 5 features


[(228, 1), (272, 1), (332, 1), (425, 1), (448, 1), (539, 1), (623, 1), (1433, 1), (1623, 1), (2286, 1), (2524, 1)]
[(0, 0.27476765805668968), (1, -0.19743979334750472), (2, -0.20869231855776826), (3, -0.15720336036785534), (4, -0.00022787635680215977)]


- 比较和第几句话相似，用LSI得到的索引接着做，并排序输出

In [28]:
sims = index[query_lsi]
sort_sims = sorted(enumerate(sims), key=lambda item: -item[1])
print (sort_sims)

[(235, 0.99861455), (256, 0.99844289), (123, 0.99668282), (151, 0.99656886), (607, 0.99640256), (606, 0.99637026), (608, 0.99634922), (54, 0.99603671), (335, 0.99581724), (483, 0.99514735), (517, 0.99508005), (480, 0.99467063), (321, 0.99456829), (437, 0.99443209), (416, 0.99439949), (356, 0.99377775), (455, 0.99371827), (548, 0.99356931), (540, 0.99348801), (56, 0.99343765), (178, 0.99325061), (210, 0.99309587), (593, 0.99288654), (229, 0.99268174), (292, 0.99267882), (345, 0.99232519), (200, 0.99206769), (579, 0.99206752), (78, 0.99178112), (31, 0.99176812), (1, 0.99174446), (27, 0.9914661), (510, 0.99145329), (460, 0.99142879), (73, 0.99140078), (494, 0.9913528), (549, 0.99124646), (529, 0.9911682), (350, 0.99115878), (103, 0.99087226), (255, 0.99068844), (427, 0.99053085), (127, 0.99042386), (351, 0.99009395), (310, 0.98975217), (168, 0.9896794), (348, 0.98965317), (269, 0.9892112), (181, 0.98889315), (526, 0.98845565), (558, 0.98839062), (105, 0.98836601), (290, 0.9882701), (129, 